In [ ]:
### Intro to ML, High school level - Practical session ###
# Alice Schoenauer Sebag, Feb. 2018
# Altschuler&Wu lab, UCSF

## First of all, let us download the data. The original dataset [1] is publicly available 
# (see below), and it was pre-processed using the method described in [2]. 
# References: 
# [1] [Caie et al., Molecular Cancer Therapeutics, 2010], [Ljosa et al., Nature Methods, 2012]
#      https://data.broadinstitute.org/bbbc/BBBC021/
# [2] [Kang et al., Nature Biotech., 2016]
#      https://www.nature.com/articles/nbt.3419

import os, pandas
import numpy as np
if not os.path.isfile('IntroToML_TD.tar.gz'):
    print('Be patient while the data downloads')
    os.system('wget -c http://members.cbio.mines-paristech.fr/~aschoenauer/IntroToML_TD.tar.gz')
    os.system('tar -xvzf IntroToML_TD.tar.gz')
    print('Done!')

In [ ]:
## We have downloaded 2 files: the X (features) as well as the Y vector (drug classes, and some additional information).
# Let us have a look at what is inside these files.
Y = pandas.read_csv('IntroToML_TD/Caie_metadata.csv')

# This will show how the file looks like: it contains what we want to predict, the DrugClass, and some additional info.
Y.head()

In [ ]:
# Let's see how many datapoints we have:
print('Nb of datapoints:',Y.shape[0])

# **Question** Can you tell me how many drug classes are in the data?
print('Nb of drug classes?') #your answer :)

In [ ]:
# Let's do the same with the feature matrix
X = pandas.read_csv('IntroToML_TD/Caie_features.csv')
X.head()

# **Question** How many features do we have?
print('Nb of features?')

In [ ]:
## Now we would like to plot the data! In a nutshell, we're going to transform it first, 
# for the plot to be more informative.

# ** Depending on your math level, read or skip this! **
# Rather than have the axes be the features, we will have the axes be combinations of features, 
# the combinations along which the data varies most. This method is called PCA, for Principal 
# Component Analysis. Never forget: before applying PCA, we need to standardize the data because 
# PCA relies on Euclidean distances between datapoints.

# To learn more about PCA, https://en.wikipedia.org/wiki/Principal_component_analysis

from sklearn.decomposition import PCA
import matplotlib.pyplot as p
from mpl_toolkits.mplot3d import Axes3D

pca = PCA(n_components=3)
# Data standardization
nX = (X-np.mean(X,0))/np.std(X, 0)
# Let us find the best projection and use it to transform the data.
npX = pca.fit_transform(nX)

# Now we can plot it.
f=p.figure(figsize=(18,18))
ax = f.gca(projection='3d')
ax.scatter(npX[:,0], npX[:,1], npX[:,2])
p.show()

In [ ]:
# This is all very well, but we would be interested to see how the drug classes are distributed in this space, not 
# only the datapoints. => Let us add colors

# Picking some colors
color_names = ['red', 'blue', 'orange', 'magenta', 'green','black', '0.5', '#bcbddc','yellow','cyan', '#a6cee3','#dd1c77', '#1f78b4', '#b2df8a', '#33a02c', '#fb9a99', '#e31a1c', '#fdbf6f', '#ff7f00', '#cab2d6', '#6a3d9a']

# These are the distinct drug classes present in the data.
distinct_drug_classes = sorted(list(Y.DrugClass.unique()))

# Picking the right color for each datapoint.
colors = []
drug_classes = Y.DrugClass.values
for drug_class in drug_classes:
    index = distinct_drug_classes.index(drug_class)
    colors.append(color_names[index])
    
# Let us plot again
f=p.figure(figsize=(18,18))
ax = f.gca(projection='3d')
ax.scatter(npX[:,0], npX[:,1], npX[:,2], color=colors)
p.show()

In [ ]:
# 3D plots can sometimes be misleading because of the perspective. Furthermore, what do you think of 
# plots without a legend? => Let us plot in 2D, add a legend and label the axes.

f, subplots=p.subplots(3,1, figsize=(15,22))
subplots[0].scatter(npX[:,0], npX[:,1], color=colors, alpha=0.5)
subplots[0].set_xlabel('Principal component 1')
subplots[0].set_ylabel('Principal component 2')
subplots[1].scatter(npX[:,0], npX[:,2], color=colors, alpha=0.5)
subplots[1].set_xlabel('Principal component 1')
subplots[1].set_ylabel('Principal component 3')
subplots[2].scatter(npX[:,1], npX[:,2], color=colors, alpha=0.5)
subplots[2].set_xlabel('Principal component 2')
subplots[2].set_ylabel('Principal component 3')
# Adding the legend
for i, drug_class in enumerate(distinct_drug_classes):
    subplots[2].scatter(0,0, color=color_names[i], label=drug_class)
subplots[2].legend()

p.show()

In [ ]:
# **Question** If you look into what is in Y, you will see that you also have the information whether a datapoint is 
# a positive control, a negative control, or an actual experiment. Can you tell the difference between the three?
# Better, can you redo the above plots, but with the colors corresponding to positive control, negative control,
# or other?

In [ ]:
## Finally, the actual classification! For this, we will retrict ourselves to the subset of the data
# whose drug class is actually known, and which show activity at the dose used.

wh = np.where((Y.DrugClass=='Unknown')|(Y.Active==False))[0]
# Deleting the useless lines in the data
active_Y = Y.drop(wh, axis=0)
active_X = np.delete(X.values, wh, 0)

print('Nb of datapoints retained: ', active_X.shape[0])

In [ ]:
# Now we are going to choose linear Support Vector Machines as our hypothesis space, ie, type of model.
from sklearn.svm import SVC
model = SVC(kernel='linear')

# Let us fit the model to the data
model.fit(active_X, active_Y.DrugClass)
predicted_Y = model.predict(active_X)

# Let us look at the error! We will compute the accuracy of the model and its confusion matrix. 
# The entry (i,j) of the confusion matrix will tell how many datapoints with true label i were predicted to be j.

from sklearn.metrics import confusion_matrix, recall_score
score = recall_score(active_Y.DrugClass, predicted_Y, average='micro')
print("Model accuracy: ", score)
conf_mat = confusion_matrix(active_Y.DrugClass, predicted_Y)
print(conf_mat)

# Pretty good! But... wait... what did we say about overfitting? Is this the right way to estimate the error
# of our model on new data??

In [ ]:
# **Question** Split the data into a training set and a test set as we discussed, and get a more accurate estimate
# of the model's accuracy on data it has never seen.